In [1]:
import datetime as dt #For inputing any date or time information
from datetime import timezone #For inputing timezones
from google.cloud import bigquery          #The BigQuery API
import matplotlib.pyplot as plt      #For plotting
import numpy as np                         #For scientific computation
import os                            #For work with native operating system and directories
from pathlib import Path             #For working with file paths and directories
import pydata_google_auth                  #For authentication against Google 
import pandas as pd                        #For data manipulation and bgq --> pandas conversion
import pandas_gbq as pd_gbq #Import BigQuery data to create a pandas dataframe
import plotly.graph_objects as go # interactive plots
import pytz #For timezone calculations
import seaborn as sns                #For plotting
import warnings

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
warnings.filterwarnings('ignore')

In [5]:
query = """
SELECT
  DISTINCT COMPANY_CODE,
  NATIONAL_ACCOUNT,
  CUSTOMER_NUMBER_AND_NAME,
  ENTERPRISE_ACCOUNT_NAME,
  CUSTOMER_NUMBER,
  BUSINESS_AREA,
  CREDIT_CONTROL_AREA,
  BUSINESS_UNIT_GROUP,
  DOCUMENT_DATE_IN_DOCUMENT,
  POSTING_DATE_IN_DOCUMENT,
  PAYMENT_DATE,
  CC_NET_DUE_DAY,
  ACCOUNTING_DOCUMENT_NUMBER,
  ACCOUNTING_DOCUMENT_LINE_ITEM,
  TERMS_OF_PAYMENT_KEY,
  PAYMENT_TERM_SALES,
  DOCUMENT_TYPE,
  CREDIT_GROUP,
  CHECK_RULE,
  RISK_CLASS_NAME,
  AR_STATUS,
  DOCUMENT_STATUS,
  PAYMENT_REASON_CODE,
  PAYMENT_REASON_CODE_TEXT,
  BILLING_DOCUMENT,
  DOCUMENT_DATE_AGING,
  REFERENCE_DOCUMENT_NUMBER,
  AMOUNT_IN_LOCAL_CURRENCY
FROM
  `edna-data-pr-cah.VW_PHM_GFSS_NP.VW_AR_AGING`
WHERE
  (DOCUMENT_TYPE IN ('ZF',
      'VF', 'DH', 'DL', 'DZ', 'DW', 'DC'))
ORDER BY
  CUSTOMER_NUMBER,
  COMPANY_CODE,
  BUSINESS_AREA,
  DOCUMENT_DATE_IN_DOCUMENT,
  ACCOUNTING_DOCUMENT_NUMBER,
  ACCOUNTING_DOCUMENT_LINE_ITEM
"""

In [6]:
# Import the data with imputation
bqclient = bigquery.Client()
brokerage_df_df = bqclient.query(query).to_dataframe()

In [7]:
## change data types to strings
brokerage_df_df["COMPANY_CODE"] = brokerage_df_df["COMPANY_CODE"].astype(str)
brokerage_df_df["CUSTOMER_NUMBER"] = brokerage_df_df["CUSTOMER_NUMBER"].astype(str)
brokerage_df_df["BUSINESS_AREA"] = brokerage_df_df["BUSINESS_AREA"].astype(str)
brokerage_df_df["BUSINESS_UNIT_GROUP"] = brokerage_df_df["BUSINESS_UNIT_GROUP"].astype(str)
brokerage_df_df["ACCOUNTING_DOCUMENT_NUMBER"] = brokerage_df_df["ACCOUNTING_DOCUMENT_NUMBER"].astype(str)
brokerage_df_df["ACCOUNTING_DOCUMENT_LINE_ITEM"] = brokerage_df_df["ACCOUNTING_DOCUMENT_LINE_ITEM"].astype(str)
brokerage_df_df["TERMS_OF_PAYMENT_KEY"] = brokerage_df_df["TERMS_OF_PAYMENT_KEY"].astype(str)
brokerage_df_df["PAYMENT_TERM_SALES"] = brokerage_df_df["PAYMENT_TERM_SALES"].astype(str)
brokerage_df_df["DOCUMENT_TYPE"] = brokerage_df_df["DOCUMENT_TYPE"].astype(str)
brokerage_df_df["CREDIT_GROUP"] = brokerage_df_df["CREDIT_GROUP"].astype(str)
brokerage_df_df["CHECK_RULE"] = brokerage_df_df["CHECK_RULE"].astype(str)
brokerage_df_df["RISK_CLASS_NAME"] = brokerage_df_df["RISK_CLASS_NAME"].astype(str)
brokerage_df_df["ENTERPRISE_ACCOUNT_NAME"] = brokerage_df_df["ENTERPRISE_ACCOUNT_NAME"].astype(str)

In [8]:
## change to pandas datetime object
brokerage_df_df['DOCUMENT_DATE_IN_DOCUMENT'] = pd.to_datetime(brokerage_df_df['DOCUMENT_DATE_IN_DOCUMENT'], format='%Y-%m-%d')
brokerage_df_df['POSTING_DATE_IN_DOCUMENT'] = pd.to_datetime(brokerage_df_df['POSTING_DATE_IN_DOCUMENT'], format='%Y-%m-%d')
brokerage_df_df['PAYMENT_DATE'] = pd.to_datetime(brokerage_df_df['PAYMENT_DATE'], format='%Y-%m-%d')
brokerage_df_df['CC_NET_DUE_DAY'] = pd.to_datetime(brokerage_df_df['CC_NET_DUE_DAY'], format='%Y-%m-%d')

In [25]:
brokerage_df_df.PAYMENT_REASON_CODE_TEXT.unique()

array([None, 'UR - UNIDENTIFIED RESIDUAL',
       'RN - Remit Needed (Unapplied Cash)',
       'RQ - Remit Questions (Unapplied Cash)', 'AC - ALREADY CLEARED',
       'CA - CASH ON ACCOUNT (includes CASH IN ADVNC)',
       'OP - Over Payment', 'WTO - WO - Tolerance',
       'UD - DISCOUNT (EARNED/UNEARNED)', 'RZE - Customer Rebates',
       'RB - REBATE ISSUE', 'TX - TAXES', 'QT - QUANTITY',
       'JRF - Journal - Refund (CAH and Non CAH)',
       'IN - INVOICE Not Received,COPY NEEDED',
       'ED - EDI TRANSMISSION ERROR',
       'IP - Installment Plan (Not Due to Dispute)',
       'CA - CASH ON ACCOUNT (NOTEincludes CASH IN AD',
       'VP - Value+ Dispute/Resid', 'NF - NON SUFFICIENT FUNDS',
       'PI - PRICING Issue', 'WBF - WO Bank Fee', 'BF - BANK FEE',
       'PO - INCORRECT PURCHASE ORDER ENTRY', 'BK - BANKRUPTCY',
       'NC - Non Cardinal Funds',
       'CU - Customer Unidentified (Unapplied Cash)',
       'RZA - GPO Pass-Thru AC', 'TR - TRANSPORTATION/HANDLING',
       'B

In [10]:
total_ar = (
    brokerage_df_df
    .filter([ "AMOUNT_IN_LOCAL_CURRENCY", "DOCUMENT_TYPE", 'AR_STATUS', 'CUSTOMER_NUMBER'])
    .query('DOCUMENT_TYPE in ("DH", "DL", "DZ", "DW", "DC")')
    .groupby(['CUSTOMER_NUMBER', 'AR_STATUS'])
    .agg({'AMOUNT_IN_LOCAL_CURRENCY' : ['sum']})
    .sort_values(['CUSTOMER_NUMBER', 'AR_STATUS'])
    .reset_index()
)
total_ar.columns = [ 'CUSTOMER_NUMBER', 'AR_STATUS', 'CASH_FLOW'] 

In [11]:
ar_open = total_ar.query('AR_STATUS == "OPEN"')

# 1. Provide a List of customers with Total AR = $0 where there are open Transactions.

In [12]:
zero_AMOUNT_IN_LOCAL_CURRENCY = ar_open.query('CASH_FLOW == 0')

In [13]:
zero_AMOUNT_IN_LOCAL_CURRENCY

,CUSTOMER_NUMBER,AR_STATUS,CASH_FLOW
482,2050001234,OPEN,0E-9
1397,2050003661,OPEN,0E-9
4440,2051134685,OPEN,0E-9
5067,2052000713,OPEN,0E-9
5613,2052001648,OPEN,0E-9
5859,2052002128,OPEN,0E-9
7487,2052009468,OPEN,0E-9
8106,2052011680,OPEN,0E-9
9502,2052016450,OPEN,0E-9
12666,2052027258,OPEN,0E-9


# 2. List of customers with Payment amount = Invoice Amount but not cleared

In [16]:
match_me = (
    brokerage_df_df
    .filter([ "POSTING_DATE_IN_DOCUMENT", "AMOUNT_IN_LOCAL_CURRENCY", "PAYMENT_REASON_CODE",'CUSTOMER_NUMBER'])
    # .query('DOCUMENT_TYPE in ("DH", "DL", "DZ", "DW", "DC")')
    .groupby(["POSTING_DATE_IN_DOCUMENT",'CUSTOMER_NUMBER', "PAYMENT_REASON_CODE"])
    .agg({'AMOUNT_IN_LOCAL_CURRENCY' : ['sum']})
    .sort_values(["POSTING_DATE_IN_DOCUMENT"])
    .reset_index()
)
match_me.columns = ['DATE','CUSTOMER_NUMBER', "PAYMENT_REASON_CODE", 'CASH_FLOW']

In [18]:
match_me.CUSTOMER_NUMBER.value_counts()

4900000285    768
4400100373    699
2057185893    618
4400008232    605
4400100370    576
             ... 
2057216368      1
2057207348      1
2057188776      1
2057193547      1
2052025361      1
Name: CUSTOMER_NUMBER, Length: 44962, dtype: int64

In [23]:
temp = match_me.query('CUSTOMER_NUMBER == "4900000285" and "2023-08-31" >= DATE >= "2023-08-01"')

In [27]:
temp.sort_values('CASH_FLOW')

,DATE,CUSTOMER_NUMBER,PAYMENT_REASON_CODE,CASH_FLOW
4915979,2023-08-09,4900000285,CU,-627529.600000000
4999245,2023-08-15,4900000285,CU,-208014.230000000
4935788,2023-08-10,4900000285,CU,-188319.810000000
5183170,2023-08-28,4900000285,CU,-132292.090000000
5099619,2023-08-22,4900000285,CU,-99041.660000000
4851876,2023-08-04,4900000285,CU,-84750.140000000
4955665,2023-08-11,4900000285,CU,-83909.720000000
4896236,2023-08-08,4900000285,CU,-71111.240000000
5056819,2023-08-18,4900000285,CU,-60885.180000000
5118442,2023-08-23,4900000285,CU,-42580.050000000


# NFS

In [30]:
match_me.query('PAYMENT_REASON_CODE == "NF"').sort_values('CUSTOMER_NUMBER')

,DATE,CUSTOMER_NUMBER,PAYMENT_REASON_CODE,CASH_FLOW
4323794,2023-06-28,2052008101,NF,32616.070000000
4781155,2023-07-31,2052008101,NF,25236.930000000
3706260,2023-05-16,2052008597,NF,25.000000000
5603848,2023-09-26,2052008813,NF,3436.410000000
5377758,2023-09-11,2052008813,NF,2211.640000000
...,...,...,...,...
5634455,2023-09-28,4400103056,NF,26371.200000000
4175860,2023-06-19,4400103056,NF,34542.860000000
4730452,2023-07-27,4400103056,NF,5361.210000000
4316067,2023-06-28,4400103056,NF,12545.610000000


In [31]:
nf=match_me.query('CUSTOMER_NUMBER=="2052008101"')

In [33]:
nf.PAYMENT_REASON_CODE.value_counts()

      295
RN      6
UR      2
NF      2
Name: PAYMENT_REASON_CODE, dtype: int64

## 32,616.070000000

In [37]:
nf.query('"2023-05-01" <= DATE <= "2023-12-01"').sort_values('CASH_FLOW').head(50)

,DATE,CUSTOMER_NUMBER,PAYMENT_REASON_CODE,CASH_FLOW
5792754,2023-10-09,2052008101,,-59982.430000000
5377990,2023-09-11,2052008101,,-58082.680000000
3847144,2023-05-25,2052008101,,-53806.320000000
5583470,2023-09-25,2052008101,,-50370.810000000
3933404,2023-06-01,2052008101,,-46501.440000000
3624049,2023-05-10,2052008101,,-42712.060000000
4323795,2023-06-28,2052008101,,-42616.070000000
4259463,2023-06-23,2052008101,,-42578.860000000
4065356,2023-06-10,2052008101,,-42541.070000000
4452309,2023-07-07,2052008101,,-42519.000000000


# 3. List of customers with old payments which haven't cleared, Analyze the results. View if any attributes link payment to invoices.

# 4. Old residual income (Credit Balances) which match Invoice values